#1. Library imports

In [1]:
!pip install -q datasets transformers[sentencepiece]
from sklearn.metrics import confusion_matrix, classification_report

     |████████████████████████████████| 325 kB 4.3 MB/s 
     |████████████████████████████████| 3.8 MB 45.1 MB/s 
     |████████████████████████████████| 136 kB 49.7 MB/s 
     |████████████████████████████████| 212 kB 48.3 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 26.4 MB/s 
     |████████████████████████████████| 127 kB 51.4 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 144 kB 55.8 MB/s 
     |████████████████████████████████| 271 kB 47.3 MB/s 
     |████████████████████████████████| 596 kB 47.1 MB/s 
     |████████████████████████████████| 895 kB 54.3 MB/s 
     |████████████████████████████████| 6.5 MB 38.1 MB/s 
     |████████████████████████████████| 1.2 MB 44.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import tqdm
import nltk
import torch
import numpy as np
import pandas as pd
from torch import nn
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import confusion_matrix, classification_report
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import transformers

In [5]:
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import tqdm
from sklearn.metrics import confusion_matrix, classification_report

# 2. Data Imports

In [6]:
def load_data(split_name='train', columns=['text', 'stars'], folder='gdrive/MyDrive/COMP_4332/Project1/data'):
        df = pd.read_csv(f'{folder}/{split_name}.csv')
        return df

In [7]:
train_df = load_data('train', columns=['text', 'stars'])
valid_df = load_data('valid', columns=['text', 'stars'])
# the test set labels (the 'stars' column) are not available! So the following code will instead return all columns
test_df = load_data('test', columns=['text', 'stars'])

In [8]:
train_df = train_df[["text","stars"]]
train_df["stars"] = train_df["stars"]-1
valid_df = valid_df[["text","stars"]]
valid_df["stars"] = valid_df["stars"]-1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [11]:
import tempfile
import pathlib
import pyarrow as pa
import pyarrow.parquet as pq

In [12]:
import datasets
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(x):
    return tokenizer(x, truncation=True)#return_tensors = 'pt'

def tokenize_df(df):
  tokens = df['text'].map(tokenize_function)
  df['input_ids'] = [x['input_ids'] for x in tokens]
  df['attention_mask'] = [x['attention_mask'] for x in tokens]
  df['token_type_ids'] = [x['token_type_ids'] for x in tokens]
  df.rename(columns={"stars":"labels"},inplace=True)
  return df

train_df = tokenize_df(train_df)
valid_df = tokenize_df(valid_df)
test_df = tokenize_df(test_df)

small_train = train_df
small_validate = valid_df

table_train = pa.table({'labels': (list(small_train["labels"])), 'input_ids':list(small_train["input_ids"]), 'attention_mask':list(small_train["attention_mask"]),'token_type_ids':list(small_train["token_type_ids"]) })
table_validate = pa.table({'labels': list(small_validate["labels"]),'input_ids':list(small_validate["input_ids"]), 'attention_mask':list(small_validate["attention_mask"]),'token_type_ids':list(small_validate["token_type_ids"])})

#training = datasets.DatasetDict({"labels":list(train_df["labels"]), "text": list(train_df["text"])})
training = datasets.Dataset(table_train)
valid = datasets.Dataset(table_validate)

MyDataset = datasets.DatasetDict({"train":training,"validation":valid})

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [13]:
type(MyDataset['train'][0]['input_ids'])

list

# Model

In [14]:
from transformers import TrainingArguments
training_args = TrainingArguments("test-trainer")

In [15]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=5)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [16]:
type(MyDataset)

datasets.dataset_dict.DatasetDict

In [17]:
tokenized_datasets = MyDataset.with_format("torch")

In [18]:
#tokenized_datasets

In [19]:
tokenized_datasets["train"][0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'input_ids': tensor([  101,  1045,  1005,  2310,  2042,  2182,  1037,  9210,  1997,  2335,
          2085,  1998,  1045,  1005,  2310,  2196,  2042,  9364,  1012,  1996,
          2833,  2003,  2467,  2204,  1998,  1996, 14903,  2024,  4248,  1012,
          2061,  2521,  2026,  2048,  5440,  5167,  2024,  1996, 23582,  4887,
          3401, 15890,  2007,  2627,  6444,  2072,  1998,  1996, 23582,  4887,
          3401

In [20]:
tokenized_datasets["train"][0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'input_ids': tensor([  101,  1045,  1005,  2310,  2042,  2182,  1037,  9210,  1997,  2335,
          2085,  1998,  1045,  1005,  2310,  2196,  2042,  9364,  1012,  1996,
          2833,  2003,  2467,  2204,  1998,  1996, 14903,  2024,  4248,  1012,
          2061,  2521,  2026,  2048,  5440,  5167,  2024,  1996, 23582,  4887,
          3401, 15890,  2007,  2627,  6444,  2072,  1998,  1996, 23582,  4887,
          3401

In [21]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Train

In [22]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 18000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6750


Step,Training Loss
500,0.951900
1000,0.817100
1500,0.786700
2000,0.784700
2500,0.666400
3000,0.604300
3500,0.609200
4000,0.583300


Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1000
Configuration saved in test-trainer/checkpoint-1000/config.json
Model weights saved in test-trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1500
Configuration saved in test-trainer/checkpoint-1500/config.json
Model weights saved in test-trainer/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1500/tokenizer_config.json
Special tokens file saved

Step,Training Loss
500,0.951900
1000,0.817100
1500,0.786700
2000,0.784700
2500,0.666400
3000,0.604300
3500,0.609200
4000,0.583300
4500,0.560300
5000,0.389100


Saving model checkpoint to test-trainer/checkpoint-4500
Configuration saved in test-trainer/checkpoint-4500/config.json
Model weights saved in test-trainer/checkpoint-4500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-4500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-4500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-5000
Configuration saved in test-trainer/checkpoint-5000/config.json
Model weights saved in test-trainer/checkpoint-5000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-5000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-5000/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-5500
Configuration saved in test-trainer/checkpoint-5500/config.json
Model weights saved in test-trainer/checkpoint-5500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-5500/tokenizer_config.json
Special tokens file 

TrainOutput(global_step=6750, training_loss=0.59071125736943, metrics={'train_runtime': 6670.3517, 'train_samples_per_second': 8.096, 'train_steps_per_second': 1.012, 'total_flos': 9407538399320784.0, 'train_loss': 0.59071125736943, 'epoch': 3.0})

In [23]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

***** Running Prediction *****
  Num examples = 2000
  Batch size = 8


(2000, 5) (2000,)


In [ ]:
preds = np.argmax(predictions.predictions, axis=-1)

In [25]:
gt = np.array(valid_df["labels"])#[:25])

In [26]:
import sklearn
print("Validation Accuracy is",sklearn.metrics.accuracy_score(gt,preds))

print(classification_report(gt, preds))
print("\n\n")
print(confusion_matrix(gt, preds))

Validation Accuracy is 0.698
              precision    recall  f1-score   support

           0       0.86      0.80      0.83       282
           1       0.42      0.49      0.45       136
           2       0.50      0.49      0.49       212
           3       0.55      0.56      0.55       466
           4       0.83      0.82      0.82       904

    accuracy                           0.70      2000
   macro avg       0.63      0.63      0.63      2000
weighted avg       0.70      0.70      0.70      2000




[[225  44   9   2   2]
 [ 29  67  37   3   0]
 [  2  39 103  62   6]
 [  2   7  49 260 148]
 [  4   2  10 147 741]]
Validation Accuracy is 0.698
              precision    recall  f1-score   support

           0       0.86      0.80      0.83       282
           1       0.42      0.49      0.45       136
           2       0.50      0.49      0.49       212
           3       0.55      0.56      0.55       466
           4       0.83      0.82      0.82       904

    accu